In [4]:
import datetime
import time
import pymupdf as fitz  # PyMuPDF
import requests
import json
import numpy as np
from transformers import BertTokenizer
import os

# 文本分割函数

In [5]:
# 初始化BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def split_text(text, max_length, max_tokens):
    """将文本按字符数和token数分割成不超过max_length字符和max_tokens的段落"""
    paragraphs = []
    current_paragraph = ""
    current_tokens = 0

    for line in text.split("\n"):
        line_tokens = tokenizer.encode(line, add_special_tokens=False)
        if (len(current_paragraph) + len(line) + 1 <= max_length) and (current_tokens + len(line_tokens) + 1 <= max_tokens):
            current_paragraph += line + "\n"
            current_tokens += len(line_tokens) + 1  # +1 for the newline token
        else:
            paragraphs.append(current_paragraph.strip())
            current_paragraph = line + "\n"
            current_tokens = len(line_tokens) + 1

    if current_paragraph:
        paragraphs.append(current_paragraph.strip())
    return paragraphs


/root/miniconda3/envs/dev/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# 从PDF中提取文本

In [6]:
import os

# 从PDF文件中提取文本函数
def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text


# 提取PDF文件中的文本并按token数和字符数分割
pdf_files = ["20240401-0级.pdf"]
max_length = 18000
max_tokens = 4000
documents = []
for pdf in pdf_files:
    text = extract_text_from_pdf(pdf)
    paragraphs = split_text(text, max_length, max_tokens)
    documents.extend(paragraphs)

# 将分割后的文本块分别存储到多个txt文件中，并保存文件名到一个列表中
txt_dir = 'new_txt_files'
file_names = []
for idx, doc in enumerate(documents, start=1):
    file_name = os.path.join(txt_dir, f"text_file{idx}.txt")
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(doc)
    file_names.append(file_name)

print("文本文件已成功生成。")
print("生成的文件名列表：", file_names)

文本文件已成功生成。
生成的文件名列表： ['new_txt_files/text_file1.txt', 'new_txt_files/text_file2.txt', 'new_txt_files/text_file3.txt', 'new_txt_files/text_file4.txt', 'new_txt_files/text_file5.txt', 'new_txt_files/text_file6.txt', 'new_txt_files/text_file7.txt', 'new_txt_files/text_file8.txt', 'new_txt_files/text_file9.txt', 'new_txt_files/text_file10.txt', 'new_txt_files/text_file11.txt', 'new_txt_files/text_file12.txt', 'new_txt_files/text_file13.txt']


# 问题生成prompt

In [7]:
prompt1 = '''
#01 你是一个问答对数据集处理专家。

#02 你的任务是根据我给出的内容，生成适合作为问答对数据集的问题。

#03 该内容有很多从表格中提取出的内容，多根据表格内容生成问题。

#04 问题要尽量短，不要太长。

#05 一句话中只能有一个问题，问题尽量有意义。

#06 最多生成10个问题。

#07 生成问题示例：

"""

"积分视场光谱仪的0级数据包含哪些内容？"
"MCI模块的数据文件命名规范是什么样的？"
"MSC_MS 0级数据关键字PCOUNT是什么意思？"
"MSC_MS 0级数据关键字EXTVER的数据类型是什么？"

"""

#07 以下是我给出的内容：

"""

{{此处替换成你的内容}}

"""
'''

# 问答对生成prompt

In [8]:
prompt2 = '''
#01 你是一个问答对数据集处理专家。

#02 你的任务是根据我的问题和我给出的内容，生成对应的问答对。

#03 答案要全面，只使用我的信息，如果找不到答案，就回复从文档中找不到答案。

#04 你必须根据我的问答对示例格式来生成：

"""

{"content": "积分视场光谱仪的0级数据包含哪些内容？", "summary": "积分视场光谱仪包含以下内容：1.观测目标的原始曝光图像；2.参考图像， 包括偏置、 平场、 暗电流等原始图像数据；3.观测期间的原始导星数据（和多通道成像
仪共用）。这些数据格式都是FITS文件。"}

{"content": "MSC_MS 0级数据关键字PCOUNT是什么意思？", "summary": 在MSC_MS 0级数据头文件中，PCOUNT关键字表示伴随数据的参数数组的长度。在这个上下文中，对于原始数据，PCOUNT的样例值是0，表示没有伴随数据的参数数组。"}

#05 我的问题如下：

"""

{{此处替换成你上一步生成的问题}}

"""

#06 我的内容如下：

"""

{{此处替换成你的内容}}

"""
'''

# 文心一言配置

In [9]:
# 设置百度文心一言的API密钥和端点
API_KEY = "MxvHfAoOFUATRfpnohbnBAYb"
SECRET_KEY = "hOW7n2JSxNJQV1UvYxoUHmoNkmxGi3eB"

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

# 问题生成函数

In [10]:
def generate_question(text_content, more=False):
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token=" + get_access_token()
    content= "生成适合作为问答对的问题"
    if more:
        content = "尽可能多生成适合作为问答对的问题"
    prompt = prompt1.replace("{{此处替换成你的内容}}", text_content)
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": content
            }
        ],
        "temperature": 0.95,
        "top_p": 0.8,
        "system":prompt
    })
    headers = {
        'Content-Type': 'application/json'
    }
    start_time = time.time()
    response = requests.request("POST", url, headers=headers, data=payload)
    x = json.loads(response.text)
    print("耗时", time.time() - start_time)
    print(x)
    if response.status_code == 200:
        return x['result']
    else:
        print(f"Error: {response.status_code}")
        print(response.content)
        return None


# 问答对生成函数

In [11]:
def generate_qa(text_content, question_text=None):
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token=" + get_access_token()
    content= "拼成问答对"
    prompt = prompt2.replace("{{此处替换成你上一步生成的问题}}", question_text).replace("{{此处替换成你的内容}}", text_content)
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": content
            }
        ],
        "temperature": 0.95,
        "top_p": 0.8,
        "system":prompt
    })
    headers = {
        'Content-Type': 'application/json'
    }
    start_time = time.time()
    response = requests.request("POST", url, headers=headers, data=payload)
    x = json.loads(response.text)
    print("耗时", time.time() - start_time)
    print(x)
    if response.status_code == 200:
        return x['result']
    else:
        print(f"Error: {response.status_code}")
        print(response.content)
        return None


# 将生成的问答对写入.txt文件

In [12]:
# 将分割后的文本块分别存储到多个txt文件中，并保存文件名到一个列表中
def write_to_file(content):
    txt_dir = 'generated_txt_files'
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    file_name = os.path.join(txt_dir, f"new_file_{timestamp}.txt")
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(content)
    print("File 'new_file.txt' has been created and written.")

# 读取 pdf生成的txt文件

In [13]:
def read_file(file_name):
    try:
        with open(file_name, "r", encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")

In [9]:
# 生成包含文件名的列表，从'text_file1.txt'到'text_file69.txt'
file_names = [f"txt_files/text_file{i}.txt" for i in range(1, 70)]

# 打印结果以验证
print(file_names)

['txt_files/text_file1.txt', 'txt_files/text_file2.txt', 'txt_files/text_file3.txt', 'txt_files/text_file4.txt', 'txt_files/text_file5.txt', 'txt_files/text_file6.txt', 'txt_files/text_file7.txt', 'txt_files/text_file8.txt', 'txt_files/text_file9.txt', 'txt_files/text_file10.txt', 'txt_files/text_file11.txt', 'txt_files/text_file12.txt', 'txt_files/text_file13.txt', 'txt_files/text_file14.txt', 'txt_files/text_file15.txt', 'txt_files/text_file16.txt', 'txt_files/text_file17.txt', 'txt_files/text_file18.txt', 'txt_files/text_file19.txt', 'txt_files/text_file20.txt', 'txt_files/text_file21.txt', 'txt_files/text_file22.txt', 'txt_files/text_file23.txt', 'txt_files/text_file24.txt', 'txt_files/text_file25.txt', 'txt_files/text_file26.txt', 'txt_files/text_file27.txt', 'txt_files/text_file28.txt', 'txt_files/text_file29.txt', 'txt_files/text_file30.txt', 'txt_files/text_file31.txt', 'txt_files/text_file32.txt', 'txt_files/text_file33.txt', 'txt_files/text_file34.txt', 'txt_files/text_file35

# 主程序

In [15]:
file_names

['new_txt_files/text_file1.txt',
 'new_txt_files/text_file2.txt',
 'new_txt_files/text_file3.txt',
 'new_txt_files/text_file4.txt',
 'new_txt_files/text_file5.txt',
 'new_txt_files/text_file6.txt',
 'new_txt_files/text_file7.txt',
 'new_txt_files/text_file8.txt',
 'new_txt_files/text_file9.txt',
 'new_txt_files/text_file10.txt',
 'new_txt_files/text_file11.txt',
 'new_txt_files/text_file12.txt',
 'new_txt_files/text_file13.txt']

In [16]:
for file in file_names:
    text_content = read_file(file)
    print(file)
    print ('text_content\n', text_content)
    question_text = generate_question(text_content=text_content, more=False)
    print('question_text\n', question_text)
    qa_text = generate_qa(text_content=text_content, question_text=question_text)
    print('qa_text\n', qa_text)
    write_to_file(qa_text)

new_txt_files/text_file1.txt
text_content
 编    号 
KSC-00-JK-0001-03.01 
密    级 
阶段标记 
Y 
页    数 
124 
中国科学院国家天⽂台 
  中国科学院上海天⽂台 
  2024 年  3  ⽉ 20  ⽇
名  称： 
CSST 科学数据处理系统 0 级数据 
输入需求和数据结构设计说明 
编  写 
校  对 
审  核 
标  审 
批  准 
会 签 
文档修改记录
版本号
修改内容描述
修改人
修改日期备注
01.01
创建
汤静
21.12.01
02.01
修改
汤静
23.04.27
03.01
修改
汤静
24.03.27
1
目录
1.
范围.......................................................................................................... 1
1.1. 标识...................................................................................................... 1
1.2. 文档概述..............................................................................................1
2.
引用文档.................................................................................................. 1
3.
术语定义.................................................................................................. 1
4.
输入数据需求..........................................................................................2
5.
0 级数据结构接口设计和需求..........

## 将问答对保存到df中

In [17]:
import pandas as pd
import json
import glob

def clean_json_content(content):
    # 去掉开头的 'json\n[' 和结尾的 ']'
    if content.startswith("```json\n["):
        content = content[9:].strip()  # 去掉 'json\n['
    if content.endswith(']\n```'):
        content = content[:-6].strip()  # 去掉 ']'

    # 确保去掉多余的 [ 和 ]
    if content.startswith("["):
        content = content[1:].strip()
    if content.endswith("]"):
        content = content[:-1].strip()

    # 检查内容的结尾是否是完整的 {"content": ,"summary": } 对
    while content:
        try:
            json.loads(f"[{content}]")
            break  # 如果内容是完整的 JSON，则不需要删除
        except json.JSONDecodeError:
            # 如果内容不是完整的 JSON，则删除最后一个字符并继续检查
            content = content.rsplit(',', 1)[0].strip()
            continue

    return content

# 获取所有以 new_file_ 开头的 .txt 文件
file_list = glob.glob("generated_txt_files/new_file_*.txt")

# 初始化一个空列表来存储所有的 JSON 对象
json_objects = []

# 遍历每个文件并将内容合并到 json_objects 列表中
for file_name in file_list:
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            content = file.read().strip()
            cleaned_content = clean_json_content(content)  # 这里可能会抛出异常

        # 将处理后的内容解析为 JSON 对象并添加到 json_objects 列表中
        try:
            json_objects.extend(json.loads(f"[{cleaned_content}]"))
            print(file_name)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {file_name}: {e}")
            print(f"Content: {cleaned_content}")

    except Exception as e:  # 捕获 clean_json_content() 函数的异常
        print(f"Error processing file {file_name}: {e}")
        continue  # 跳过当前循环，继续执行下一个循环
    
# 转换为 DataFrame
df1 = pd.DataFrame(json_objects)

# 确保 DataFrame 只有 "content" 和 "summary" 两列
df1 = df1[['content', 'summary']]

# 检查 DataFrame 内容
print(df1)


generated_txt_files/new_file_20240731025743.txt
generated_txt_files/new_file_20240731025608.txt
generated_txt_files/new_file_20240731030524.txt
generated_txt_files/new_file_20240731030311.txt
generated_txt_files/new_file_20240731025912.txt
generated_txt_files/new_file_20240731030203.txt
generated_txt_files/new_file_20240731031116.txt
generated_txt_files/new_file_20240731030643.txt
generated_txt_files/new_file_20240731030415.txt
generated_txt_files/new_file_20240731025433.txt
generated_txt_files/new_file_20240731030951.txt
generated_txt_files/new_file_20240731030833.txt
generated_txt_files/new_file_20240731030042.txt
                               content  \
0               主巡天模块0级数据的文件夹命名结构是怎样的？   
1         MCI模块中，每个探测器单次曝光的数据存储在什么文件中？   
2         IFS模块的0级数据文件名中，CAMERA字段代表什么？   
3    CPIC模块的可见光相机和红外相机在0级数据文件名中分别如何表示？   
4        HSTDM模块中，探测器的黑体校准数据在文件名中如何标识？   
..                                 ...   
125       对于MSC_IR，DETSIZE表示什么，其数据类型是？   
126    MCI数据文件头中的IMGNUM是什么意思，其数据类型是什么？  

In [18]:
df1

,content,summary
0,主巡天模块0级数据的文件夹命名结构是怎样的？,主巡天模块0级数据的文件夹命名结构从设备总体到不同终端的不同功能层层递进，例如：CSST_L...
1,MCI模块中，每个探测器单次曝光的数据存储在什么文件中？,MCI模块中，每个探测器单次曝光（含BIAS读取等定标过程）的读出数据存储为一个FITS文件...
2,IFS模块的0级数据文件名中，CAMERA字段代表什么？,IFS模块的0级数据文件名中，CAMERA字段代表光谱仪编号，红端CCD为“R”，蓝端CCD...
3,CPIC模块的可见光相机和红外相机在0级数据文件名中分别如何表示？,CPIC模块的0级数据文件名中，可见光相机和红外相机分别用“VIS”和“NIR”表示。
4,HSTDM模块中，探测器的黑体校准数据在文件名中如何标识？,HSTDM模块中，探测器的黑体校准数据在文件名中通过TYPE2字段标识为“BLK”，如CSS...
...,...,...
125,对于MSC_IR，DETSIZE表示什么，其数据类型是？,对于MSC_IR，DETSIZE表示探测器的大小，通常以像素为单位（例如'640x512'）...
126,MCI数据文件头中的IMGNUM是什么意思，其数据类型是什么？,MCI数据文件头中的IMGNUM表示图像编号，是整数型（int），用于区分同一观测序列中的不...
127,MCI的FILTERNO指的是什么，其数据类型是怎样的？,MCI的FILTERNO指的是使用的滤光片编号，是字符串型（str），用于标识观测时所使用的...
128,CSST_MCI数据文件头中的DATE代表什么，数据类型是？,CSST_MCI数据文件头中的DATE代表文件写入的日期和时间（yyyy-mm-ddThh:...


In [19]:
df1.to_csv("QA_新0级数据.csv")